# 17: Merge Tier and Nextbike data

Here we are going to merge the tier and nextbike datasets (which are already merged with VRS)

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from shapely import Point

In [2]:
data_month_single = '7'
data_month_double = '07'

## load data

In [3]:
tier = pd.DataFrame(pd.read_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/13_VRS_Tier_Merge_Complete_NEU_NEU_TEST.parquet", engine="pyarrow"))

In [4]:
next = pd.DataFrame(pd.read_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/16_VRS_Next_Merge_Complete_NEU_NEU_TEST.parquet", engine="pyarrow"))

In [5]:
# transform geometry
tier_point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["station_point"], crs=4326))
tier_buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["buffer_zone"], crs=4326))

In [6]:
# drop geometry columns in wrong format
tier = tier.drop(columns=["station_point", "buffer_zone"])

In [7]:
# replace geometry columns with correct format
tier['station_point'] = tier_point
tier['buffer_zone'] = tier_buffer

In [8]:
# transform geometry
next_point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(next["station_point"], crs=4326))
next_buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(next["buffer_zone"], crs=4326))

In [9]:
# drop geometry columns in wrong format
next = next.drop(columns=["station_point", "buffer_zone"])

In [10]:
# replace geometry columns with correct format
next['station_point'] = next_point
next['buffer_zone'] = next_buffer

## Merge Tier and Next Dataframes

In [11]:
tier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873991 entries, 0 to 873990
Data columns (total 35 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Unnamed: 0.2                     873991 non-null  int64         
 1   Unnamed: 0.1                     873991 non-null  int64         
 2   Unnamed: 0                       873991 non-null  int64         
 3   route_id                         873991 non-null  int64         
 4   agency_id                        873991 non-null  int64         
 5   route_short_name                 873991 non-null  object        
 6   route_type                       873991 non-null  int64         
 7   route_type_name                  873991 non-null  object        
 8   agency_name                      873991 non-null  object        
 9   service_id                       873991 non-null  int64         
 10  trip_id                          873991 non-

In [12]:
tier.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

In [13]:
tier

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,service,date,weekday,transfer_stop,tier_trips_count,tier_trips_id,tier_trips_end_at_station_count,tier_trips_end_at_station_id,station_point,buffer_zone
0,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,...,1,2023-07-03,Monday,True,0,,0,[],POINT (7.04787 50.73740),"POLYGON ((7.05062 50.73773, 7.05066 50.73756, ..."
1,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-01,Saturday,False,0,,0,[],POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ..."
2,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-02,Sunday,False,0,,0,[],POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ..."
3,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-01,Saturday,False,0,,0,[],POINT (7.08113 50.73656),"POLYGON ((7.08388 50.73690, 7.08392 50.73672, ..."
4,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-02,Sunday,False,0,,0,[],POINT (7.08113 50.73656),"POLYGON ((7.08388 50.73690, 7.08392 50.73672, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873986,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,False,0,,0,[],POINT (7.09765 50.73628),"POLYGON ((7.10040 50.73662, 7.10044 50.73644, ..."
873987,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,True,0,,0,[],POINT (7.09487 50.73377),"POLYGON ((7.09762 50.73411, 7.09766 50.73393, ..."
873988,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,True,0,,0,[],POINT (7.10248 50.73747),"POLYGON ((7.10523 50.73780, 7.10527 50.73763, ..."
873989,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,True,0,,0,[],POINT (7.11744 50.73923),"POLYGON ((7.12019 50.73957, 7.12023 50.73939, ..."


In [14]:
next.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873991 entries, 0 to 873990
Data columns (total 35 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   Unnamed: 0.2                         873991 non-null  int64         
 1   Unnamed: 0.1                         873991 non-null  int64         
 2   Unnamed: 0                           873991 non-null  int64         
 3   route_id                             873991 non-null  int64         
 4   agency_id                            873991 non-null  int64         
 5   route_short_name                     873991 non-null  object        
 6   route_type                           873991 non-null  int64         
 7   route_type_name                      873991 non-null  object        
 8   agency_name                          873991 non-null  object        
 9   service_id                           873991 non-null  int64         
 

In [15]:
next.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

In [16]:
next

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,service,date,weekday,transfer_stop,nextbike_trips_count,nextbike_trips_id,nextbike_trips_end_at_station_count,nextbike_trips_end_at_station_id,station_point,buffer_zone
0,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,26965,1-18-001-697.1.13:034800-39-256_56C5151C-E402-...,Buchheim Herler Str.,0,...,1,2023-07-03,Monday,True,0,,0,[],POINT (7.04787 50.73740),"POLYGON ((7.05062 50.73773, 7.05066 50.73756, ..."
1,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-01,Saturday,False,0,,0,[],POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ..."
2,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-02,Sunday,False,0,,0,[],POINT (7.09713 50.73201),"POLYGON ((7.09988 50.73234, 7.09992 50.73217, ..."
3,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-01,Saturday,False,0,,0,[],POINT (7.08113 50.73656),"POLYGON ((7.08388 50.73690, 7.08392 50.73672, ..."
4,18,1,18,0,Tram,KVB Kölner Verkehrs-Betriebe AG,27521,10-18-001-687.4.13:253800-50-2_8D415A25-8BCC-4...,Thielenbruch,0,...,1,2023-07-02,Sunday,False,0,,0,[],POINT (7.08113 50.73656),"POLYGON ((7.08388 50.73690, 7.08392 50.73672, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873986,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,False,0,,0,[],POINT (7.09765 50.73628),"POLYGON ((7.10040 50.73662, 7.10044 50.73644, ..."
873987,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,True,0,,0,[],POINT (7.09487 50.73377),"POLYGON ((7.09762 50.73411, 7.09766 50.73393, ..."
873988,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,True,0,,0,[],POINT (7.10248 50.73747),"POLYGON ((7.10523 50.73780, 7.10527 50.73763, ..."
873989,55,1,SB55,3,Bus,KVB Kölner Verkehrs-Betriebe AG,11577,999-55-001-687.2.81:193500-23-257_D9298D62-2E6...,Lülsdorf Nord,0,...,1,2023-07-02,Sunday,True,0,,0,[],POINT (7.11744 50.73923),"POLYGON ((7.12019 50.73957, 7.12023 50.73939, ..."


In [17]:
merge = pd.merge(tier, next, on=['route_id', 'agency_id', 'route_short_name', 'route_type',
       'route_type_name', 'agency_name', 'service_id', 'trip_id',
       'trip_headsign', 'direction_id', 'shape_id', 'stop_id',
       'actual_arrival_time', 'actual_departure_time', 'vrs_timestamp',
       'stop_sequence', 'stop_headsign', 'stop_name', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'departure_delay',
       'service', 'date', 'weekday', 'transfer_stop', 'station_point', 'buffer_zone'], how="outer", suffixes=('_tier', '_next'))

In [18]:
merge

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,tier_trips_count,tier_trips_id,tier_trips_end_at_station_count,tier_trips_end_at_station_id,station_point,buffer_zone,nextbike_trips_count,nextbike_trips_id,nextbike_trips_end_at_station_count,nextbike_trips_end_at_station_id
0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.07351 50.73910),"POLYGON ((7.07626 50.73943, 7.07630 50.73926, ...",0,,0,[]
1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.15955 50.68392),"POLYGON ((7.16230 50.68425, 7.16234 50.68408, ...",0,,0,[]
2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.15835 50.68034),"POLYGON ((7.16110 50.68067, 7.16113 50.68050, ...",0,,0,[]
3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.11452 50.72175),"POLYGON ((7.11727 50.72208, 7.11731 50.72191, ...",0,,0,[]
4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.11158 50.72556),"POLYGON ((7.11433 50.72589, 7.11437 50.72572, ...",0,,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873986,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]
873987,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]
873988,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]
873989,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]


In [19]:
gdf = gpd.GeoDataFrame(merge, geometry='station_point')

In [20]:
gdf.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/17_VRS_Tier_Next_Merge_Complete_NEU_NEU_TEST.parquet", engine="pyarrow")

In [21]:
gdf

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,tier_trips_count,tier_trips_id,tier_trips_end_at_station_count,tier_trips_end_at_station_id,station_point,buffer_zone,nextbike_trips_count,nextbike_trips_id,nextbike_trips_end_at_station_count,nextbike_trips_end_at_station_id
0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.07351 50.73910),"POLYGON ((7.07626 50.73943, 7.07630 50.73926, ...",0,,0,[]
1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.15955 50.68392),"POLYGON ((7.16230 50.68425, 7.16234 50.68408, ...",0,,0,[]
2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.15835 50.68034),"POLYGON ((7.16110 50.68067, 7.16113 50.68050, ...",0,,0,[]
3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.11452 50.72175),"POLYGON ((7.11727 50.72208, 7.11731 50.72191, ...",0,,0,[]
4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],POINT (7.11158 50.72556),"POLYGON ((7.11433 50.72589, 7.11437 50.72572, ...",0,,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873986,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]
873987,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]
873988,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]
873989,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",0,,0,[]
